##Preparation: We load in the necessary libraries

In [1]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##  1. Parsing of Speech Text into Individual Sentences

In [97]:
# We load the CSV containing the scraped speeches, last update January 2023 (all speeches in 2022 included)
data_fed = pd.read_csv("/content/drive/MyDrive/FED_Press_Project_Colab_Data/Data_Input/Raw_FED_Data_update_2022.csv", encoding="latin-1")
data_ecb = pd.read_csv("/content/drive/MyDrive/CentralBankBERTa/Data/ECB/all_ECB_speeches.csv", sep='|')
data_bis = pd.read_csv("/content/drive/MyDrive/CentralBankBERTa/Data/BIS/all_speeches_BIS.csv")

In [98]:
# We clean BIS data for FED and ECB data
banks = ['european monetary institute', 
         'european central bank', 
         'central bank of cyprus', 
         'board of governors of the federal reserve system',
         'central bank of ireland',
         'bank of italy',
         'bank of slovenia',
         'bank of estonia',
         'bank of france',
         'deutsche bundesbank',
         'bank of spain',
         'federal reserve bank of new york', 
         'netherlands bank',
         'central bank of the republic of austria',
         'bank of greece',
         'central bank of malta',
         'national bank of belgium',
         'bank of lithuania',
         'central bank of hungary',
         'central bank of iceland',
         'bank of portugal',
         'national bank of slovakia', 
         'bank of latvia',
         'central bank of luxembourg',
         'federal reserve bank of dallas',
         'federal reserve bank of philadelphia',
         'federal reserve bank of richmond',
         'federal reserve bank of atlanta', 
         'federal reserve bank of chicago',
         'federal reserve bank of minneapolis',
         'federal reserve bank of san francisco',
         'federal reserve bank of kansas city', 'central bank of belize',
         'federal reserve bank of boston',
         'european monetary institute']
data_BIS = data_BIS[~data_BIS['institution'].isin(banks)]
data_BIS['institution'].unique()

array(['czech national bank', 'central bank of seychelles',
       'central bank of the philippines', 'south african reserve bank',
       'reserve bank of india', nan, 'national bank of denmark',
       'monetary authority of singapore', 'bank of england',
       'national bank of serbia', 'reserve bank of new zealand',
       'central bank of malaysia', 'reserve bank of australia',
       'bank of japan', 'hong kong monetary authority',
       'central bank of norway', 'bulgarian national bank',
       'bank of canada', 'bank of jamaica', 'bank of albania',
       'croatian national bank', 'bank of mexico',
       'eastern caribbean central bank', 'bank of korea',
       'bank of finland', 'bank of thailand', 'bank indonesia',
       'national bank of ukraine',
       'central bank of the republic of kosovo', 'central bank of chile',
       'bank of zambia', 'central bank of sweden', 'swiss national bank',
       'bank of mauritius', 'bank of israel', "people's bank of china",
      

In [ ]:
# We give every speech a doc_id between 1 and 6764
for i in range(0, len(data_ecb)): 
  data_ecb['doc_id'][i] = int(i)+1

In [99]:
data_ecb = data_ecb[data_ecb['contents'].notna()]
data_ecb

,date,speakers,title,subtitle,contents
1,2023-02-16,Philip R. Lane,The euro area hiking cycle: an interim assessment,"Dow Lecture by Philip R. Lane, Member of the E...",SPEECH The euro area hiking cycle: an inter...
2,2023-02-16,Fabio Panetta,Monetary policy after the energy shock,"Speech by Fabio Panetta, Member of the Executi...",SPEECH Monetary policy after the energy sho...
3,2023-02-15,Christine Lagarde,European Parliament plenary debate on the ECB ...,"Speech by Christine Lagarde, President of the ...",SPEECH European Parliament plenary debate ...
5,2023-01-23,Christine Lagarde,New challenges in a changing world,"Speech by Christine Lagarde, President of the ...",SPEECH New challenges in a changing world ...
6,2023-01-23,Fabio Panetta,"The digital euro: our money wherever, whenever...","Introductory statement by Fabio Panetta, Membe...",SPEECH The digital euro: our money whereve...
...,...,...,...,...,...
2656,1997-05-13,Alexandre Lamfalussy,The European Central Bank: independent and acc...,Keynote speech delivered by Alexandre Lamfalus...,The European Central Bank: independent and a...
2657,1997-04-30,Alexandre Lamfalussy,The operation of monetary policy in stage thre...,"Address by Alexandre Lamfalussy, President of ...",The operation of monetary policy in stage th...
2658,1997-04-22,Alexandre Lamfalussy,Convergence and the role of the European Centr...,"Remarks by Alexandre Lamfalussy, President of ...",Convergence and the role of the European Cen...
2659,1997-03-10,Alexandre Lamfalussy,Securing the benefits of EMU,"Address by Alexandre Lamfalussy, President of ...",Securing the benefits of EMU Address by Al...


In [105]:
data_bis = data_bis[data_bis['content'].notna()]
data_bis

,ID,page,page_url,row_number,date,title,author,info,url,url_type,content,pdf_link,institution
0,1,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,0,17 Mar 2023,Jan Frait: Monetary macroeconomics and central...,Jan Frait,"Lecture by Mr Jan Frait, Deputy Governor of th...",https://www.bis.org/review/r230315l.htm,htm,I'd like to thank the conference organisers fo...,https://www.bis.org/review/r230315l.pdf,czech national bank
1,2,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,1,16 Mar 2023,Christine Lagarde: ECB press conference - intr...,Christine Lagarde,Introductory statement by Ms Christine Lagarde...,https://www.bis.org/review/r230316h.htm,htm,"Good afternoon, the Vice-President and I welco...",https://www.bis.org/review/r230316h.pdf,european central bank
2,3,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,2,16 Mar 2023,Caroline Abel: Financial inclusion policies fo...,Caroline Abel,"Opening remarks (virtual) by Ms Caroline Abel,...",https://www.bis.org/review/r230316b.htm,htm,"Distinguished Guests,Participants,Greetings to...",https://www.bis.org/review/r230316b.pdf,central bank of seychelles
3,4,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,3,16 Mar 2023,Jenifer Sullivan: Welcome - 28th Information a...,Jenifer Sullivan,"Welcome by Ms Jenifer Sullivan, Second Deputy ...",https://www.bis.org/review/r230316a.htm,htm,"Mr Francis Selialia, Head of CCBG SecretariatM...",https://www.bis.org/review/r230316a.pdf,central bank of seychelles
4,5,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,4,15 Mar 2023,Constantinos Herodotou: Financial literacy and...,Constantinos Herodotou,"Welcome address by Mr Constantinos Herodotou, ...",https://www.bis.org/review/r230315j.htm,htm,Your Excellency member of the Parliament Dr Ma...,https://www.bis.org/review/r230315j.pdf,central bank of cyprus
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17193,18480,Page 740 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,4,08 Jan 1997,Ms. Rivlin discusses the prudential regulation...,0,Remarks by the Vice Chairman of the Board of G...,https://www.bis.org/review/r970108b.pdf,pdf,Ms. Rivlin discusses the prudential regulation...,https://www.bis.org/review/r970108b.pdf,board of governors of the federal reserve system
17194,18481,Page 740 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,5,08 Jan 1997,Mr. Meyer examines the role for structural mac...,0,"Remarks by Mr. Laurence H. Meyer, a member of ...",https://www.bis.org/review/r970108a.pdf,pdf,Mr. Meyer examines the role for structural mac...,https://www.bis.org/review/r970108a.pdf,board of governors of the federal reserve system
17195,18482,Page 740 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,6,07 Jan 1997,Bank of Japan's December review of monetary an...,0,"BANK OF JAPAN, MONTHLY ECONOMIC REVIEW, 20/12/96.",https://www.bis.org/review/r970107b.pdf,pdf,Bank of Japan's December review of monetary an...,https://www.bis.org/review/r970107b.pdf,bank of japan
17196,18483,Page 740 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,7,07 Jan 1997,Mr. Matsushita considers the role of monetary ...,0,Translated excerpts of a speech given by the G...,https://www.bis.org/review/r970107a.pdf,pdf,Mr. Matsushita considers the role of monetary ...,https://www.bis.org/review/r970107a.pdf,bank of japan


In [106]:
text_ecb = data_ecb['contents']
text_ecb = text_ecb.astype('string')
text_bis = data_bis['content']
text_bis = text_bis.astype('string')

In [107]:
data_bis.head()

,ID,page,page_url,row_number,date,title,author,info,url,url_type,content,pdf_link,institution
0,1,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,0,17 Mar 2023,Jan Frait: Monetary macroeconomics and central...,Jan Frait,"Lecture by Mr Jan Frait, Deputy Governor of th...",https://www.bis.org/review/r230315l.htm,htm,I'd like to thank the conference organisers fo...,https://www.bis.org/review/r230315l.pdf,czech national bank
1,2,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,1,16 Mar 2023,Christine Lagarde: ECB press conference - intr...,Christine Lagarde,Introductory statement by Ms Christine Lagarde...,https://www.bis.org/review/r230316h.htm,htm,"Good afternoon, the Vice-President and I welco...",https://www.bis.org/review/r230316h.pdf,european central bank
2,3,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,2,16 Mar 2023,Caroline Abel: Financial inclusion policies fo...,Caroline Abel,"Opening remarks (virtual) by Ms Caroline Abel,...",https://www.bis.org/review/r230316b.htm,htm,"Distinguished Guests,Participants,Greetings to...",https://www.bis.org/review/r230316b.pdf,central bank of seychelles
3,4,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,3,16 Mar 2023,Jenifer Sullivan: Welcome - 28th Information a...,Jenifer Sullivan,"Welcome by Ms Jenifer Sullivan, Second Deputy ...",https://www.bis.org/review/r230316a.htm,htm,"Mr Francis Selialia, Head of CCBG SecretariatM...",https://www.bis.org/review/r230316a.pdf,central bank of seychelles
4,5,Page 1 of 740,https://www.bis.org/cbspeeches/index.htm?cbspe...,4,15 Mar 2023,Constantinos Herodotou: Financial literacy and...,Constantinos Herodotou,"Welcome address by Mr Constantinos Herodotou, ...",https://www.bis.org/review/r230315j.htm,htm,Your Excellency member of the Parliament Dr Ma...,https://www.bis.org/review/r230315j.pdf,central bank of cyprus


In [103]:
# We put every speech in a parser. Speeches the parser cannot handle are skipped (skipped speech # are printed). 
# We give every parsed sentence a number ("Sentence_Rank") that indicates the position it occured in the speech. 
ecb_parsed = pd.DataFrame(columns = ["Parsed_Text", "Speaker", "Date"])
index = 0 
count = 1 
for i in text_ecb:
    r = sent_tokenize(i)
    if r == []: 
        print("we skipped speech #", index)
        count = count + 1
    else:
        ecb_parsed_intermediate = pd.DataFrame(r, columns = ["Parsed_Text"] )
        ecb_parsed_intermediate.loc[:,"Speaker"] = data_ecb['speakers'].iloc[index]
        ecb_parsed_intermediate.loc[:,"Date"] = data_ecb['date'].iloc[index]
        index_list = []
        for k in range(0, len(ecb_parsed_intermediate)): 
          index_list.append(k)
        ecb_parsed_intermediate['Sentence_Rank'] = index_list
        ecb_parsed = pd.concat([ecb_parsed, ecb_parsed_intermediate], ignore_index=True, sort=False)
    index = index + 1 
ecb_parsed

we skipped speech # 2101


,Parsed_Text,Speaker,Date,Sentence_Rank
0,SPEECH The euro area hiking cycle: an inter...,Philip R. Lane,2023-02-16,0.0
1,Christopher Dow had a distinguished career as ...,Philip R. Lane,2023-02-16,1.0
2,"[1] Moreover, he extensively analysed my topic...",Philip R. Lane,2023-02-16,2.0
3,[2] While the current inflation environment is...,Philip R. Lane,2023-02-16,3.0
4,My aim today is to provide an interim analysis...,Philip R. Lane,2023-02-16,4.0
...,...,...,...,...
343652,My feeling is that you will be able to avoid s...,Alexandre Lamfalussy,1997-02-07,161.0
343653,"But for a long time to come, most of your cust...",Alexandre Lamfalussy,1997-02-07,162.0
343654,The comparative assessment of the costs and be...,Alexandre Lamfalussy,1997-02-07,163.0
343655,"I hope that in ten years' time, if I am still ...",Alexandre Lamfalussy,1997-02-07,164.0


In [109]:
bis_parsed = pd.DataFrame(columns = ["Parsed_Text", "Speaker", "Date"])
index = 0 
count = 1 
for i in text_bis:
    r = sent_tokenize(i)
    if r == []: 
        print("we skipped speech #", index)
        count = count + 1
    else:
        bis_parsed_intermediate = pd.DataFrame(r, columns = ["Parsed_Text"] )
        bis_parsed_intermediate.loc[:,"Speaker"] = data_bis['author'].iloc[index]
        bis_parsed_intermediate.loc[:,"Date"] = data_bis['date'].iloc[index]
        index_list = []
        for k in range(0, len(bis_parsed_intermediate)): 
          index_list.append(k)
        bis_parsed_intermediate['Sentence_Rank'] = index_list
        bis_parsed = pd.concat([bis_parsed, bis_parsed_intermediate], ignore_index=True, sort=False)
    index = index + 1 
bis_parsed

,Parsed_Text,Speaker,Date,Sentence_Rank
0,I'd like to thank the conference organisers fo...,Jan Frait,17 Mar 2023,0.0
1,The knowledge I gained from my discussions her...,Jan Frait,17 Mar 2023,1.0
2,"At conferences, central bankers these days ten...",Jan Frait,17 Mar 2023,2.0
3,"But for this conference, for the first time in...",Jan Frait,17 Mar 2023,3.0
4,This will hopefully better enable me to convey...,Jan Frait,17 Mar 2023,4.0
...,...,...,...,...
1757101,"Needless\nto say, that is just as true today, ...",0,06 Jan 1997,119.0
1757102,But the fact that the Bundesbank isattested a ...,0,06 Jan 1997,120.0
1757103,That verdict has been confirmed time and again...,0,06 Jan 1997,121.0
1757104,Hence theBundesbank will also continue to feel...,0,06 Jan 1997,122.0


##  2. Text Cleaning 

In [110]:
# We use Regex to remove special characters
ecb_parsed['Parsed_Text'] = ecb_parsed['Parsed_Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
bis_parsed['Parsed_Text'] = bis_parsed['Parsed_Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [111]:
# We remove all sentences that are shorter than 20 characters
ecb_parsed = ecb_parsed[ecb_parsed['Parsed_Text'].str.len()>=20] 
bis_parsed = bis_parsed[bis_parsed['Parsed_Text'].str.len()>=20] 

In [ ]:
# Remove common redundant strings
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('cid173', '', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('digitized for fraser httpfraserstlouisfedorg federal reserve bank of st louis', '', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('UFB03', 'ffi', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('U0080U0099', '', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('U0080U0091', '', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('U0080U0094', '', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('U0080', '', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('U0099', '', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('U0093', '', x))
df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('U009', '', x))

<ipython-input-156-aa035e354bde>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('cid173', '', x))
<ipython-input-156-aa035e354bde>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Parsed_Text'] = df['Parsed_Text'].apply(lambda x: re.sub('digitized for fraser httpfraserstlouisfedorg federal reserve bank of st louis', '', x))
<ipython-input-156-aa035e354bde>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [112]:
# We make all text lowercase
ecb_parsed['Parsed_Text'] = ecb_parsed['Parsed_Text'].map(str.lower)
bis_parsed['Parsed_Text'] = bis_parsed['Parsed_Text'].map(str.lower)

<ipython-input-112-2dee4672d0dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecb_parsed['Parsed_Text'] = ecb_parsed['Parsed_Text'].map(str.lower)
<ipython-input-112-2dee4672d0dc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bis_parsed['Parsed_Text'] = bis_parsed['Parsed_Text'].map(str.lower)


In [113]:
# We remove all dublicate sentences 
ecb_parsed = ecb_parsed.drop_duplicates(subset=['Parsed_Text'])
bis_parsed = bis_parsed.drop_duplicates(subset=['Parsed_Text'])

In [114]:
ecb_parsed = ecb_parsed.reset_index()
bis_parsed = bis_parsed.reset_index()

In [115]:
ecb_parsed = ecb_parsed.drop('index', axis=1)
bis_parsed = bis_parsed.drop('index', axis=1)

In [119]:
# We save the data
ecb_parsed.to_csv("/content/drive/MyDrive/CentralBankBERTa/Data/Pre_Processed_ECB_Data.csv")
bis_parsed.to_csv("/content/drive/MyDrive/CentralBankBERTa/Data/Pre_Processed_BIS_Data.csv")
# The pre-processing is finished

## 3. NOT RELEVANT - We extract a 2% random sample for manual labelling 

In [ ]:
#sample = data.sample(frac = 0.02)
#sample.to_csv("sampled_data.csv")